In [1]:
# fold
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
import keras
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# For some reason I have to tell it to use TensorFlows dimension ordering
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from time import time
import os
import tensorflow as tf
from keras import applications
from keras.utils import multi_gpu_model
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.visible_device_list = "3"
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

# Globals
VERBOSE = 0
ARCHITECTURE = 0
NORMALISE = 4096
# Class 0 = backgrounds
# Server scratch
CLASSDIR_0 = '/scratch/wm0015/prem/Segments/'
CLASSDIR_1 = '/scratch/wm0015/prem/2D/4mm/'
CLASSDIR_1 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download3/2D/4mm/'
# Prem..
# CLASSDIR_0 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/Segments/'#P_Tomo_6CMBCF_XYZ_5*'
# CLASSDIR_1 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/2D/6mm/'#P_6CMBCF*'
# CLASSDIR_0 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/Segments/'#P_Tomo_6CMBCF_XYZ_5*'
# CLASSDIR_1 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/2D/my/6mm_c95'#P_6CMBCF*

# CLASSDIR_0 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download2/4AFC-study-data/BgTomoFiltered_visualcheck_6mm'
# CLASSDIR_1 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download2/4AFC-study-data/Final_dataset/2D/6mm'
# CLASSDIR_0 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/crops256/malignant/1.2.840.113619.2.227.2079*'
# CLASSDIR_1 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/crops256/malignant/1.2.840.113619.2.227.2079*'
# CLASSDIR_0 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/crops256/normal/*'
# CLASSDIR_1 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/crops256/malignant/*'

# Calcs
# CLASSDIR_0 = '/user/HS204/wm0015/student/allCalcs/0/*'
# CLASSDIR_1 = '/user/HS204/wm0015/student/allCalcs/1/*'
MODEL_SAVE = '/vol/vssp/cvpwrkspc01/scratch/wm0015/models/best_model.h5'
BATCH_SIZE = 100
EPOCHS = 100
TRAIN_RATIO = 1-(118/1062)*2
INPUT_SHAPE = [429, 429, 3]
#INPUT_SHAPE = [256, 256, 3]
#INPUT_SHAPE = [385, 385, 3]
FOLDS = 9
PATIENCE = 1000

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='/vol/vssp/mammo2/will/logs/new', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        fCount=0
        while os.path.exists(os.path.join(log_dir, 'training' + '_' + str(fCount))):
            fCount+=1
        training_log_dir = os.path.join(log_dir, 'training' + '_' + str(fCount))
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation' + '_' + str(fCount))

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

    
        
def getSensitivityCallback(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())
def getSpecificityCallback(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())        

def getBitDepth(img):
    print('Bit depth: img.shape: ', img.shape)
    print('img.shape: ', img.shape)
    maxmax = 0
    count = 0
    for _ in img:
        count += 1
        print(np.amax(_), ' ', count)
        if np.amax(_) > maxmax:
            maxmax = np.amax(_)
    return maxmax

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x

def fourCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE, data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    #model.add(Conv2D(32, (3,3), activation='relu'))   
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model
    
def fiveCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE, data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  
    return model
    
def bigCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE, data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(64, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(124, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(256, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(512, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  
    return model

def vgg():
    vggModel = applications.VGG19(weights = 'imagenet', include_top=False, input_shape = INPUT_SHAPE)
    # Add custom final layer
    model = vggModel.output
    model = Flatten()(model)
    model = Dense(2, activation='softmax')(model)
    model = keras.models.Model(inputs=vggModel.input, outputs=model)
    return model

def tlVGG(train_data, val_data, test_data):
    print('Compute bottleneck features...')
    vggModel = applications.VGG19(weights = "imagenet", include_top=False, input_shape = INPUT_SHAPE)
    # Freeze all layers
    for layer in vggModel.layers:
        layer.trainable = False
    # Add custom final layer
    bNModel = vggModel.output
    bNModel = Flatten()(bNModel)
    final_model = keras.models.Model(inputs=vggModel.input, outputs=bNModel)
    train_bNFeatures = {'img': 0, 'label': train_data['label']}
    val_bNFeatures = {'img': 0, 'label': val_data['label']}
    test_bNFeatures = {'img': 0, 'label': test_data['label'], 'fileName': list(test_data['fileName'])}
    train_bNFeatures['img'] = final_model.predict(train_data['img'], batch_size=16)
    val_bNFeatures['img'] = final_model.predict(val_data['img'], batch_size=16)
    test_bNFeatures['img'] = final_model.predict(test_data['img'], batch_size=16)
    #Undo one hot - ROC does not work with onehot
    val_bNFeatures.update({'labelIndex': np.where(val_data['label']==1)[1]})
    test_bNFeatures.update({'labelIndex': np.where(test_data['label']==1)[1]})
    print('train_bNFeatures[img].shape = ', train_bNFeatures['img'].shape)
    print('val_bNFeatures[img].shape = ', val_bNFeatures['img'].shape)

    print('Train head...')
    head = Sequential()
    #head.add(Dense(32, input_dim=train_bNFeatures['img'].shape[1], activation='relu'))
    head.add(Dense(2, activation='softmax'))    
    return head, train_bNFeatures, val_bNFeatures, test_bNFeatures

    # Why oh why are they in a directory structure like this
def getPremFiles(imgPath, dataSpecs):
    import pydicom
    from fnmatch import fnmatch
    # First get all the 6mm lesions
    fileList = []
    for path, subdirs, files in os.walk(imgPath):
        for name in files:
            if fnmatch(name, '2D_dim2d.dcm'):
                fileList.append(os.path.join(path, name))
    # I can't remember why I thought dataSpecs was a good idea
    # I suppose this means that class 0 needs to be loaded in first
    dataSpecs['classLength'].append(len(fileList))
    
    # Load the files from filelist into an array
    print(len(fileList), ' Files found')
    print('Loading images...')
    dicomImg = np.asarray([])
    count = 0
    for f in fileList:
        dicomImg = np.append(dicomImg, pydicom.dcmread(f).pixel_array)
        count += 1
        print(count, '/', len(fileList))
    return dicomImg

def getFileName(filepath, levels=1):
    common = filepath
    for i in range(levels + 1):
        common = os.path.dirname(common)
    return os.path.relpath(filepath, common)


def getDicomImages(data, paths, strMatch = '*.dcm', verbose = True):  
    print('Getting dicom images')
    import pydicom
    from fnmatch import fnmatch
    # Get file paths of images
    tmp = [[],[]]
    tmpName = [[],[]]
    for index, p in enumerate(paths):
        fileList = []
        print(p)
        for path, subdirs, files in os.walk(p):
            for name in files:
                if fnmatch(name, strMatch): # '2D_dim2d.dcm'
                    fileList.append(os.path.join(path, name))
                    tmpName[index].append(getFileName(os.path.dirname(os.path.dirname(fileList[-1])), 0))
                    #print(tmpName[index][-1])
        data['classLength'].append(len(fileList))
        if len(fileList) == 0:
            print('MY_ERROR: NO FILES FOUND')
        # Load the files from filelist into an array
        print(len(fileList), ' Files found')
        print('Loading images...')
        data['img'] = np.asarray([])
        count = 0
        for f in fileList:
            tmp[index] = np.append(tmp[index], pydicom.dcmread(f).pixel_array)
            count += 1
            if verbose == True:
                print(count, '/', len(fileList))
        
    # Reshape the images
    tmp[0] = tmp[0].reshape((-1, INPUT_SHAPE[0], INPUT_SHAPE[1], 1))
    tmp[0] = tmp[0][0:data['classLength'][1],:,:,:] # Balance the dataset
    tmpName[0] = tmpName[0][0:data['classLength'][1]] # Balance the dataset
    tmp[1] = tmp[1].reshape((-1, INPUT_SHAPE[0], INPUT_SHAPE[1], 1))
    
    # Update class length
    data['classLength'][0] = tmp[0].shape[0]
    data['classLength'][1] = tmp[1].shape[0]
    
    # Concatenate the classes
    print('tmp[0].shape: ', tmp[0].shape)
    print('tmp[1].shape: ', tmp[1].shape)
    data['img'] = np.concatenate((tmp[0],tmp[1]))
    data['fileName'] = np.concatenate((tmpName[0], tmpName[1]))
    # Add the channels
    print('data[Img].shape = ', data['img'].shape)
    data['img'] = np.concatenate((data['img'], data['img'], data['img']), axis = 3)
    print('data[Img].shape = ', data['img'].shape)
    return data

# For non dicom images
def getImages(data, paths):    
    print('Getting non-dicom images')
    tmp = [[],[]]
    for index, p in enumerate(paths):
        fileList = glob.glob(p) #'BengaliBMPConvert/*.bmp' 
        data['classLength'].append(len(fileList))
        tmp[index] = np.array([(cv2.imread(fname)) for fname in fileList])
    print('tmp[0].shape: ', tmp[0].shape)
    print('tmp[1].shape: ', tmp[1].shape)
    data['img'] = np.concatenate((tmp[0], tmp[1]))
    return data

def getImagesOld(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

    # Comment out for prem images
#     data['img'] = np.concatenate((
#             get_images(CLASSDIR_0, dataSpecs), # Class 0 (backgrounds)
#             get_images(CLASSDIR_1, dataSpecs) # Class 1 
#     ))         


def colourDrop3to1(data):   
    data['img'] = data['img'][:,:,:,0]
    data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
    print('new data shape = ', data['img'].shape)
    return data

def getSensitivity(y_true, y_score):
    y_score = np.rint(y_score)
    tn, fp, fn, tp = confusion_matrix(y_true, y_score, labels=[0,1]).ravel()
#     print('Sensitivity:\ny_true:\n', y_true, '\ny_score:\n', y_score)
#     print(' tn: ', tn, ' fp: ', fp, ' fn: ', fn, ' tp: ', tp)
    return tp/(tp+fn)
def getSpecificity(y_true, y_score):
    y_score = np.rint(y_score)
    tn, fp, fn, tp = confusion_matrix(y_true, y_score, labels=[0,1]).ravel()
#     print('Specificity:\ny_true:\n', y_true, '\ny_score:\n', y_score)
#     print(' tn: ', tn, ' fp: ', fp, ' fn: ', fn, ' tp: ', tp)
    return tn/(tn+fp)

# The split is based on the fold number so that, with enough folds, every
# image will have been used in the test set
def splitDataset4AFC(data, fold, includeTestSet = True):
    print('data[img].shape', data['img'].shape)
    if includeTestSet == True:       
        splitRatio = TRAIN_RATIO    
        numImages = data['img'].shape[0]
        numTestImages = int(numImages * ((1-splitRatio)/2))
        splitPointTest = int((fold + 1) * (numImages * ((1-splitRatio)/2)))
        testMask = np.asarray(range(numTestImages)) + fold * numTestImages
        test_data = {'img': data['img'][splitPointTest - numTestImages: splitPointTest], 
                     'label': data['label'][splitPointTest - numTestImages:splitPointTest], 
                     'fileName': data['fileName'][splitPointTest - numTestImages:splitPointTest]}
        # Create shuffle mask out of remaining indexes
        shuffleMask = np.arange(numImages)
        # Remove test indexes from shuffleMask
        tmpTestMask = np.full((numImages), True)
        tmpTestMask[testMask] = False
        shuffleMask = shuffleMask[tmpTestMask]
        # Set val and train data
        np.random.shuffle(shuffleMask)
        tmpdata = dict(data)
        tmpdata['img'] = data['img'][shuffleMask]
        tmpdata['label'] = data['label'][shuffleMask]
        tmpdata['fileName'] = data['fileName'][shuffleMask]
        splitPointVal = numTestImages
        val_data = {'img': tmpdata['img'][0:splitPointVal], 'label': tmpdata['label'][0:splitPointVal]}
        train_data = {'img': tmpdata['img'][splitPointVal:], 'label': tmpdata['label'][splitPointVal:]}


        print('splitPointVal: ', splitPointVal)
        print('splitPointTest: ', splitPointTest)
                                
        print('val_data[img].shape: ', val_data['img'].shape)
        print('test_data[img].shape: ', test_data['img'].shape)
        print('train_data[img].shape', train_data['img'].shape)
        print('val_data[label].shape: ', val_data['label'].shape)
        print('test_data[label].shape: ', test_data['label'].shape)        
        print('train_data[label].shape: ', train_data['label'].shape)
        #Undo one hot - ROC does not work with onehot
        val_data.update({'labelIndex': np.where(val_data['label']==1)[1]})
        test_data.update({'labelIndex': np.where(test_data['label']==1)[1]})
        return train_data, val_data, test_data        

def splitDataset(data, seed = None, includeTestSet = True):
    if includeTestSet == True:
        # Shuffle data
        if seed != None:
            np.random.seed(seed) # Has to be set before each use of random
        shuffleMask = np.random.permutation(data['img'].shape[0]) 
        tmpdata = dict(data)
        tmpdata['img'] = data['img'][shuffleMask, :, :, :]
        tmpdata['label'] = data['label'][shuffleMask, :]
        # Split traing and validation data        
        splitRatio = TRAIN_RATIO
        splitPointVal = math.floor(tmpdata['img'].shape[0]*(1-splitRatio)/2)
        splitPointTest = math.floor(tmpdata['img'].shape[0]*(1-splitRatio))
        val_data = {'img': tmpdata['img'][0:splitPointVal], 'label': tmpdata['label'][0:splitPointVal]}
        test_data = {'img': tmpdata['img'][splitPointVal:splitPointTest], 'label': data['label'][splitPointVal:splitPointTest]}
        train_data = {'img': tmpdata['img'][splitPointTest:], 'label': tmpdata['label'][splitPointTest:]}

        print('val_data[img].shape: ', val_data['img'].shape)
        print('test_data[img].shape: ', test_data['img'].shape)
        print('train_data[img].shape', train_data['img'].shape)
        print('val_data[label].shape: ', val_data['label'].shape)
        print('test_data[label].shape: ', test_data['label'].shape)        
        print('train_data[label].shape: ', train_data['label'].shape)
        #Undo one hot - ROC does not work with onehot
        val_data.update({'labelIndex': np.where(val_data['label']==1)[1]})
        test_data.update({'labelIndex': np.where(test_data['label']==1)[1]})
        return train_data, val_data, test_data
    else:
        # Shuffle data
        if setSeed != None:
            np.random.seed(seed) # Has to be set before each use of random
        shuffleMask = np.random.permutation(data['img'].shape[0]) 
        tmpdata['img'] = data['img'][shuffleMask, :, :, :]
        tmpdata['label'] = data['label'][shuffleMask, :]
        # Split traing and validation data        
        splitRatio = TRAIN_RATIO
        splitPoint = math.floor(tmpdata['img'].shape[0]*splitRatio)
        train_data = {'img': tmpdata['img'][0:splitPoint], 'label': tmpdata['label'][0:splitPoint]}
        val_data = {'img': tmpdata['img'][splitPoint:], 'label': tmpdata['label'][splitPoint:]}
        print('TrainLabels.shape: ', train_data['label'].shape)
        print('val_data.shape: ', val_data['label'].shape)
        #Undo one hot - ROC does not work with onehot
        val_data.update({'labelIndex': np.where(val_data['label']==1)[1]})
        return train_data, val_data
    
    
    

    
def main():
    import keras
    print('keras version: ', keras.__version__)
    print('TensorFlow version: ', tf.__version__)
    print('\nLoading images...')
    
    # Get images
    data = {'img': 0, 'label': 0, 'fileName': [], 'classLength':[]}
    #data = getImages(data, (CLASSDIR_0, CLASSDIR_1))
    #data = getDicomImages(data, (CLASSDIR_0, CLASSDIR_1))
    data = getDicomImages(data, (CLASSDIR_0, CLASSDIR_1), strMatch = '2D_dim2d.dcm', verbose = False)
    
    # Create one hot labels
    labels0 = get_labels_one_hot(2, 0, data['classLength'][0])
    labels1 = get_labels_one_hot(2, 1, data['classLength'][1])
    print('labels0.shape: ', labels0.shape)
    print('labels1.shape: ', labels1.shape)
    data['label'] = np.concatenate((labels0, labels1))
    
    print('data[label].shape: ', data['label'].shape)
    # Get bit depth
#     print('Bit depth: ', getBitDepth(data['img']))
#     return
    # Normalise
    print('data[img].dtype: ', data['img'].dtype)
    data['img'] = data['img']/NORMALISE 
    print('data[img].dtype: ', data['img'].dtype)
    
    #data = colourDrop3to1(data)
     
    # After this things get a bit messy
    #___________________________________________________



    
    valStats = {'score':[], 'specificity':[], 'sensitivity':[]}
    exportNames = []
    exportConfidenceCancer = []
    for crossVal in range(FOLDS):
        successful = False
        while not successful:

            #train_data, val_data, test_data = splitDataset(data, seed = 9)
            train_data, val_data, test_data = splitDataset4AFC(data, crossVal, includeTestSet = True)



            model, train_data, val_data, test_data = tlVGG(train_data, val_data, test_data)
            #model = fiveCNN()
            #model = multi_gpu_model(model, gpus=2)
            if ARCHITECTURE != 0:
                model.summary()  

            sgd = optimizers.SGD(lr=5, decay=1e-6, momentum=0.9, nesterov=True)
            adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #0.001
            model.compile(loss='binary_crossentropy',
                        optimizer=adam,
                         metrics=['accuracy'])#, getSensitivityCallback, getSpecificityCallback])
            tensorboard = TensorBoard(log_dir='/vol/vssp/mammo2/will/logs/new'.format(time()), write_images=True)

            # Data augmentation settings
            if (1 == 2):
                from keras.preprocessing.image import ImageDataGenerator
                trainDatagen = ImageDataGenerator(
                    #featurewise_center=True,
                    #featurewise_std_normalization=True,
                    rotation_range=20,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    horizontal_flip=True)
                trainDatagen.fit(train_data['img'])
                valDatagen = ImageDataGenerator(
                    featurewise_center=True,
                    featurewise_std_normalization=True,
                    #rotation_range=20,
                    #width_shift_range=0.2,
                    #height_shift_range=0.2,
                    #horizontal_flip=True
                    )
                valDatagen.fit(val_data['img'])

    #         # Normalise myself
    #         mean_data = np.mean(train_data['img'])
    #         std_data = np.std(train_data['img'])
    #         train_data['img'] = (train_data['img']-mean_data)/std_data
    #         val_data['img'] = (val_data['img']-mean_data/std_data)

    #        # Train, with data augmentation
    #         print('Train...')
    #         model.fit_generator(trainDatagen.flow(train_data['img'], train_data['label'], batch_size=32),
    #                     steps_per_epoch=len(train_data['img']) / 32, epochs=EPOCHS, verbose=VERBOSE,
    #                            callbacks=[
    #                                TrainValTensorBoard(write_graph=False),
    #                                EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=1),
    #                                ModelCheckpoint(filepath=MODEL_SAVE, monitor='val_accuracy', save_best_only=True)],
    #                            validation_data=valDatagen.flow(x = val_data['img'], y = val_data['label']))





            # Train
            print('Train...')
            model.fit(train_data['img'], train_data['label'], 
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE,
                    callbacks=[
                        TrainValTensorBoard(write_graph=False),
                        EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=1),
                        ModelCheckpoint(filepath=MODEL_SAVE, monitor='val_acc', save_best_only=True)],
                    validation_data=(val_data['img'], val_data['label']))

            # Restore best model
            #model.load_weights(MODEL_SAVE)  
            # Test for convergence (acc > 70%)
            score = model.evaluate(val_data['img'], val_data['label'], verbose=0)  
            if score[1] < 0.70:
                print('             Model DID NOT converge: ', score[1]*100, '%')
            else:
                successful = True
                print('             Model converged: ', score[1]*100, '%')

                # Produce ROC curve
                eval_data = test_data
                y_pred_keras = model.predict(eval_data['img'])  
                y_true = eval_data['labelIndex']
                y_score = y_pred_keras[:,1]
                fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_true, y_score)
                from sklearn.metrics import auc
                auc_keras = auc(fpr_keras, tpr_keras)
                plt.figure(1)
                plt.plot([0, 1], [0, 1], 'k--')
                plt.plot(fpr_keras, tpr_keras, label='DLMO (area = {:.3f})'.format(auc_keras))
                plt.xlabel('False positive rate')
                plt.ylabel('True positive rate')
                plt.title('ROC curve')
                plt.legend(loc='best')
                plt.show()
                exportNames.append(test_data['fileName'])
                exportConfidenceCancer.append(y_score)




                # Evaluate
                score = model.evaluate(eval_data['img'], eval_data['label'], verbose=0)
                valStats['score'].append(score)
                print('The score is......\n', score)
                valStats['sensitivity'].append(getSensitivity(y_true, y_score))
                valStats['specificity'].append(getSpecificity(y_true, y_score))
                print('Sensitivity: ', valStats['sensitivity'][-1])
                print('Specificity: ', valStats['specificity'][-1])
  

    print('ecc.shape: ', np.asarray(exportConfidenceCancer).shape )
    print('en.shape: ', np.asarray(exportNames).shape)
    print('\n\nexportNames')
    for i in exportNames:
        for _ in i:
            print(_)
    print('\n\nexportConfidenceCancer:')
    for i in exportConfidenceCancer:
        for _ in i:
            print(_)
    valStatsScore = np.asarray(valStats['score'])
    print('\nValidations: \n', valStatsScore[:, 1])
    print('Average loss: ', sum(valStatsScore[:, 0])/FOLDS)
    print('Average validation: ', sum(valStatsScore[:, 1])/FOLDS)
    print('\n\nSensitivities: \n', valStats['sensitivity'])
    print('\nSpecificities: \n', valStats['specificity'])
    print('\n\nAverage sensitivity: ', sum(valStats['sensitivity'])/len(valStats['sensitivity']))
    print('Average specificity: ', sum(valStats['specificity'])/len(valStats['specificity']))
    
if __name__ == "__main__":
    main()

Using TensorFlow backend.


keras version:  2.2.0
TensorFlow version:  1.7.1

Loading images...
Getting dicom images
/scratch/wm0015/prem/Segments/
1176  Files found
Loading images...
/vol/vssp/cvpwrkspc01/scratch/wm0015/download3/2D/4mm/
554  Files found
Loading images...
tmp[0].shape:  (554, 429, 429, 1)
tmp[1].shape:  (554, 429, 429, 1)
data[Img].shape =  (1108, 429, 429, 1)
data[Img].shape =  (1108, 429, 429, 3)
labels0.shape:  (554, 2)
labels1.shape:  (554, 2)
data[label].shape:  (1108, 2)
data[img].dtype:  float64
data[img].dtype:  float64
data[img].shape (1108, 429, 429, 3)
splitPointVal:  123
splitPointTest:  123
val_data[img].shape:  (123, 429, 429, 3)
test_data[img].shape:  (123, 429, 429, 3)
train_data[img].shape (862, 429, 429, 3)
val_data[label].shape:  (123, 2)
test_data[label].shape:  (123, 2)
train_data[label].shape:  (862, 2)
Compute bottleneck features...
train_bNFeatures[img].shape =  (862, 86528)
val_bNFeatures[img].shape =  (123, 86528)
Train head...
Train...
             Model converged:  86

/user/HS204/wm0015/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


<Figure size 640x480 with 1 Axes>

/user/HS204/wm0015/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:364: RuntimeWarning: invalid value encountered in long_scalars


...

<Figure size 640x480 with 1 Axes>

The score is......
 [0.3059240972123495, 0.8780487770956706]
Sensitivity:  0.8780487804878049
Specificity:  nan
data[img].shape (1108, 429, 429, 3)
splitPointVal:  123
splitPointTest:  1108
val_data[img].shape:  (123, 429, 429, 3)
test_data[img].shape:  (123, 429, 429, 3)
train_data[img].shape (862, 429, 429, 3)
val_data[label].shape:  (123, 2)
test_data[label].shape:  (123, 2)
train_data[label].shape:  (862, 2)
Compute bottleneck features...
train_bNFeatures[img].shape =  (862, 86528)
val_bNFeatures[img].shape =  (123, 86528)
Train head...
Train...
             Model converged:  87.80487867874828 %


<Figure size 640x480 with 1 Axes>

The score is......
 [0.3026067500676566, 0.8455284567383247]
Sensitivity:  0.8455284552845529
Specificity:  nan
ecc.shape:  (9, 123)
en.shape:  (9, 123)


exportNames
P_Tomo_6CMBCF_XYZ_663_296_292_XYD_624_321_MD_16_RD_14_SDR_1_SD_2_LD_2
P_Tomo_6CMBCF_XYZ_1261_204_292_XYD_1273_221_MD_18_RD_18_SDR_1_SD_2_LD_2
P_Tomo_6CMBCH_XYZ_913_270_292_XYD_895_293_MD_23_RD_30_SDR_1_SD_2_LD_2
P_Tomo_6CMBCM_XYZ_1537_526_292_XYD_1572_570_MD_19_RD_21_SDR_1_SD_2_LD_2
P_Tomo_6CMBCG_XYZ_571_434_292_XYD_524_471_MD_28_RD_30_SDR_1_SD_2_LD_2
P_Tomo_6CMBCF_XYZ_1439_428_292_XYD_1466_464_MD_14_RD_13_SDR_1_SD_2_LD_2
P_Tomo_6CMBCF_XYZ_1439_244_292_XYD_1466_265_MD_16_RD_15_SDR_1_SD_2_LD_2
P_Tomo_6CMBCH_XYZ_1603_408_292_XYD_1643_442_MD_22_RD_22_SDR_1_SD_2_LD_2
P_Tomo_6CMBCH_XYZ_1301_382_292_XYD_1316_414_MD_21_RD_22_SDR_1_SD_2_LD_2
P_Tomo_6CMBCF_XYZ_1347_520_292_XYD_1366_564_MD_13_RD_12_SDR_1_SD_2_LD_2
P_Tomo_6CMBCH_XYZ_657_428_292_XYD_618_464_MD_26_RD_24_SDR_1_SD_2_LD_2
P_Tomo_6CMBCH_XYZ_1347_428_292_XYD_1366_464_MD_19

0.27322695
0.010728865
0.83046275
0.981935
0.3904789
0.945841
0.9961119
0.9992918
0.99947804
0.9353611
0.041668586
0.9696134
0.97604465
0.9316392
0.23772784
0.9522638
0.99934214
0.99986374
0.9501355
0.9408678
0.9622322
0.97229517
0.9582905
0.981764
0.92416024
0.45900685
0.99978405
0.99018073
0.9674884
0.9923373
0.93208176
0.745066
0.9671491
0.57075
0.9796428
0.75419545
0.9766987
0.8870341
0.9886359
0.9996904
0.9985526
0.98941094
0.8637902
0.98510695
0.9961205
0.99009097
0.3284032
0.7432637
0.73931706
0.98745674
0.8627657
0.830258
0.9565794
0.97460824
0.56009495
0.9467252
0.94113076
0.9970137
0.9773488
0.9989844
0.98158896
0.9288946
0.9962696
0.9647633
0.99957734
0.67433244
0.9999746
0.96683985
0.7791014
0.14703208
0.9897564
0.39244822
0.87414485
0.9972088
0.98904246
0.8669117
0.99080443
0.9097111
0.91568834
0.9897175
0.9652111
0.9962012
0.99907184
0.8941656
0.52917004
0.99572635
0.87007976
0.8693549
0.96643007
0.84994805
0.8131694
0.84382343
0.91087294
0.09349141
0.98384523
0.9410326
0

In [1]:
print('hello world')

hello world


In [28]:
CLASSDIR_0 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/Segments'
CLASSDIR_1 = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/2D/6mm'

def test(x):
    for _ in x:
        print(_)
test((CLASSDIR_0, CLASSDIR_1))

/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/Segments
/vol/vssp/cvpwrkspc01/scratch/wm0015/download/prem/2D/6mm


In [35]:
x = [[6.5308382717179665, 0.5925925925925926], [0.16470135086112553, 0.9444444444444444], [0.15422181794304907, 0.9444444444444444], [0.19875195217721256, 0.9382716049382716]] 
x = np.asarray(x)
print(x, '\n')
print(x[0][0])
print(x[:, 0])
print(x.shape)

[[6.53083827 0.59259259]
 [0.16470135 0.94444444]
 [0.15422182 0.94444444]
 [0.19875195 0.9382716 ]] 

6.5308382717179665
[6.53083827 0.16470135 0.15422182 0.19875195]
(4, 2)
